In [ ]:
import portal
import glob
import joblib
from joblib import Parallel, delayed
import pcmsc

%load_ext autoreload
%autoreload 2

In [ ]:
doi = "F7NG4NS1"
title = "Oceanographic time-series measurements from the mouth of the Columbia River, Oregon and Washington, 2013"
summary = """This USGS data release presents oceanographic time-series measurements of currents, waves, water-column properties, and sediment transport recorded at three locations by instrumented tripods deployed in the mouth of the Columbia River (MCR) between May and June, 2013 (USGS Field Activity S-03-13-WO). Each tripod was equipped with similar instrumentation (figure 1) including an upward looking 1200 kHz Teledyne RD Instruments acoustic Doppler current profiler (ADCP), a 5 MHz Sontek acoustic Doppler velocimeter (ADV) and Hydra data recorder, an RBR conductivity, temperature, and depth (CTD) sensor, and Imagenex 881 (fan beam) and 881A (pencil beam) sonars. 

The tripods were deployed on May 9, 2013 from the F/V Cape Windy at three locations within the MCR. The three tripods frames were approximately 2.5 m tall and 3 m wide. After navigating to the planned tripod-deployment locations, the tripods were lowered to the seafloor using the ship’s boom. Once on the bottom, the locations were recorded using a hand-held GPS and the water depth was recorded using the ship’s echosounder (table 1). Two independent methods for recovery of the tripods were included on the tripods. A ground line that led to a 500-lb clump weight and a surface buoy was the primary recovery method.  The tripods also included Edgetech CART acoustic releases attached to a 14” float and line bucket mounted on the tripod. On June 15, 2013, all the tripods were successfully recovered from the F/V Cape Windy resulting in a 38-day deployment. Two of the tripods (North and West) were recovered using the ground line and surface float. During recovery of the South tripod, the line between the clump weight and surface float parted. The acoustic release was triggered and the float and line was used to recover the instrument package.

After recovery of the tripods, the manufacturers’ software was used to download the data, apply calibration coefficients, and convert the data to scientific units. These output files were then converted by custom, instrument-specific Matlab programs to netCDF files. Digitial files containing time-series data available for each tripod are provided in .zip files. The .zip archives contain separate files with current- and wave-data from the ADCP, files with burst data and burst-averaged data from the ADV, a single file from the CTD, single files from the fan- and pencil-beam sonars, and a comma-separated text file describing the contents of the zip archive.  """

In [ ]:
%%time
files = glob.glob(f'../pcmsc/doi-{doi}/*.nc')
n = 1
for f in files:
    if "_fan" in f:
        continue
    if "_pen" in f:
        continue
    print(n/len(files)*100, f)
    c = pcmsc.convert(f, doi, title, summary)
    print(c)
    n += 1

In [ ]:
files = glob.glob(f'../pcmsc/doi-{doi}/clean/*.nc')
n = 1
for f in files:
    print(n/len(files)*100, f)
    cc = portal.check_compliance_system(f, "cf:1.6")
    print(cc.stdout.decode('utf-8'))
#     cc = portal.check_compliance_system(f, "acdd")
#     print(cc.stdout.decode('utf-8'))
    n += 1

In [ ]:
%%time
files = glob.glob('../pcmsc/doi-' + doi + '/*.nc')
# need to specify multiprocessing so we get print statements
Parallel(n_jobs=8, backend='multiprocessing')(delayed(convert)(f) for f in files)